In [4]:
! pip3 install elasticsearch --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [39]:
import pandas as pd

In [53]:
# create documents for elasticsearch
df = pd.read_csv("../data/metadata_with_episode_dates.csv")

In [73]:
from elasticsearch import Elasticsearch, helpers
INDEX_NAME = "search-spotify-dataset-visualizer"
with open("spotify-dataset-visualization.csv", "r") as fp:
    key = fp.readlines()

client = Elasticsearch(
"https://527db13b131f4f1b90590b6f6c0039f6.us-central1.gcp.cloud.es.io:443",
  # api_key=key[0]
  # api_key="UDFJMVBJd0JRcERJVVMzblQ1RDA6UE1KSnNjbGhUdnU0dGlJR3RzaEdWZw=="
  api_key="STFJVlc0d0JRcERJVVMzblRzTG86OHBDcEUzLTNUeHVENUdISmxmRlZ2dw=="
)

client.info()

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': '527db13b131f4f1b90590b6f6c0039f6', 'cluster_uuid': 'c5RsPX1ZRNeYPFZkw-JhLQ', 'version': {'number': '8.11.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6f9ff581fbcde658e6f69d6ce03050f060d1fd0c', 'build_date': '2023-11-11T10:05:59.421038163Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [11]:
df[df['episode_filename_prefix']=='1S9X55ZcDVk5AgA2nG0yuz']

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,release_date
19696,spotify:show:7Kf4MwDBFH2QXaLZadac7W,BOSSY Bruja Prayers,Our prayer line has a new home! This channel w...,Ayodele Fuega,['en'],https://anchor.fm/s/990caa0/podcast/rss,spotify:episode:1S9X55ZcDVk5AgA2nG0yuz,Love Invocation 03: Love Transforms,What’s good witches? Welcome to day 3 of our L...,5.72255,show_7Kf4MwDBFH2QXaLZadac7W,1S9X55ZcDVk5AgA2nG0yuz,2019-06-04


In [54]:
# adding topics
import pickle
with open('all_inferenced_topics.pkl', 'rb') as fp:
    episode_topics = pickle.load(fp)

df_topics = pd.DataFrame(episode_topics)
df_topics.columns = ['episode_filename_prefix', 'topics']
df_topics['episode_filename_prefix'] = df_topics['episode_filename_prefix'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [55]:
df = pd.merge(df_topics, df, on='episode_filename_prefix', how='inner')


In [58]:
from collections import Counter

def string_histogram(string_list):
    counts = Counter(string_list)
    return counts

df['topics_count'] = df['topics'].apply(string_histogram)

agg_df = df.groupby('show_uri')['topics_count'].sum().reset_index()

# merged_df = df.merge(agg_df, on='show_uri', suffixes=('', '_group'), how='inner')

# merged_df

merged_df = df.merge(agg_df, on='show_uri', how='left')

merged_df.rename(columns={'topics_count_x': 'topics_count_episode', 'topics_count_y': 'topics_count_show'}, inplace=True)

df = merged_df




In [68]:
# df.drop(['topics_x'], axis=1, inplace=True)
# df.drop(['topics_y'], axis=1, inplace=True)

In [75]:
import pandas as pd
import altair as alt
from collections import Counter
import json

charts = []

def generate_topics_show_chart(counter):
    keys = list(counter.keys())
    values = list(counter.values())
    
    chart = alt.Chart(pd.DataFrame({'Key': keys, 'Value': values})).mark_bar().encode(
        x=alt.X('Key:N', title='Key'),
        y=alt.Y('Value:Q', title='Value')
    ).properties(
        title=f'Show Topics Count'
    )

    return json.dumps(chart.to_dict())

df['topics_count_show_chart'] = df['topics_count_show'].apply(generate_topics_show_chart)


In [84]:
# deletes all records in elasticsearch
client.delete_by_query(index=[INDEX_NAME], body={"query": {"match_all": {}}})

ObjectApiResponse({'took': 5571, 'timed_out': False, 'total': 105360, 'deleted': 105360, 'batches': 106, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [76]:
df = df.fillna("")

In [78]:
# df.drop(['topics_count_episode', 'topics_count_show'], axis=1, inplace=True)

In [81]:
df['topics_count_show_chart'].iloc[0]

'{"config": {"view": {"continuousWidth": 300, "continuousHeight": 300}}, "data": {"name": "data-ab57db7ce3afd3c12a486e050db8c4a4"}, "mark": {"type": "bar"}, "encoding": {"x": {"field": "Key", "title": "Key", "type": "nominal"}, "y": {"field": "Value", "title": "Value", "type": "quantitative"}}, "title": "Show Topics Count", "$schema": "https://vega.github.io/schema/vega-lite/v5.15.1.json", "datasets": {"data-ab57db7ce3afd3c12a486e050db8c4a4": [{"Key": "diaries_&_daily_life", "Value": 14}, {"Key": "relationships", "Value": 6}, {"Key": "other_hobbies", "Value": 1}]}}'

In [86]:
# prepare bulk data
actions = [
    {
        "_op_type": "index", # change to "update" when updating existing documents
        "_index": INDEX_NAME,
        "_id": row['episode_uri'],
        "doc": row.drop('episode_uri').drop('release_date').to_dict()
    }
    for index, row in df.iterrows()
]

In [87]:
actions[0]

{'_op_type': 'index',
 '_index': 'search-spotify-dataset-visualizer',
 '_id': 'spotify:episode:1S9X55ZcDVk5AgA2nG0yuz',
 'doc': {'episode_filename_prefix': '1S9X55ZcDVk5AgA2nG0yuz',
  'topics': ['diaries_&_daily_life', 'relationships'],
  'show_uri': 'spotify:show:7Kf4MwDBFH2QXaLZadac7W',
  'show_name': 'BOSSY Bruja Prayers',
  'show_description': 'Our prayer line has a new home! This channel will be the home of BOSSYbruja’s prayer invocations and guided meditations. I pray you enjoy! Be sure to check out our Podcast “BOSSY Bruja Podcast!” Enjoy.  Support this podcast: https://anchor.fm/bossybrujaprayers/support',
  'publisher': 'Ayodele Fuega',
  'language': "['en']",
  'rss_link': 'https://anchor.fm/s/990caa0/podcast/rss',
  'episode_name': 'Love Invocation 03: Love Transforms',
  'episode_description': 'What’s good witches? Welcome to day 3 of our LOVE INVOCATION. So, y’all know I be sitting around waiting for Spirit to tell me what the topic is, right? [Laughter] Today’s topic is L

In [88]:
# bulk operation on the documents
from elasticsearch.helpers import BulkIndexError

try:
    helpers.bulk(client, actions)
except BulkIndexError as e:
    print("An error occurred during bulk indexing:")
    print(e)

    # Optionally, print more detailed information about each error
    for error in e.errors:
        print(error)

In [19]:
client.search(index=INDEX_NAME, q="inside")

ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.190946, 'hits': [{'_index': 'search-spotify-dataset-visualizer', '_id': 'spotify:episode:001UfOruzkA3Bn1SPjcdfa', '_score': 1.190946, '_source': {'doc': {'show_uri': 'spotify:show:6vZRgUFTYwbAA79UNCADr4', 'show_name': 'Inside The 18 : A Podcast for Goalkeepers by Goalkeepers', 'show_description': 'Inside the 18 is your source for all things Goalkeeping! Each week we are joined by guests from around the world. We\xa0recap the weeks events,\xa0discuss new training techniques\xa0and have candid conversations with professional goalkeepers and goalkeeper coaches. The show is a must listen for the goalkeeping enthusiast!    #insidethe18 #goalkeeperpodcast #thegoalkeepers #goalkicks #ederson #NWSL #ashlynharris #UWSNT #MLS #USMNT #goalkeepercoaches #degea #neuer #navas #areola #oblak #USL #goalkeepersaves #court

In [ ]:
client.q

In [55]:
query = {
    "query": {
        "query_string": {
            "query": "show_name:Kream in your Koffee"
        }
    }
}

response = client.search(index=INDEX_NAME, body=query)

In [54]:
response

ObjectApiResponse({'took': 52, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 18.550373, 'hits': [{'_index': 'search-spotidy-dataset', '_id': 'spotify:episode:000A9sRBYdVh66csG2qEdj', '_score': 18.550373, '_source': {'doc': {'show_uri': 'spotify:show:2NYtxEZyYelR6RMKmjfPLB', 'show_name': 'Kream in your Koffee', 'show_description': 'A 20-something blunt female takes on the world and gives you her take on it. Enjoy visits from special guests and friends to give insight and input into interesting situations.', 'publisher': 'Katie Houle', 'language': "['en']", 'rss_link': 'https://anchor.fm/s/11b84b68/podcast/rss', 'episode_name': '1: It’s Christmas Time!', 'episode_description': 'On the first ever episode of Kream in your Koffee, Katie talks about tips for Christmas shopping. We also get a little insight into who and what we’ll be hearing about in next weeks episode! ', 'duration'

[('spotify-podcasts-2020/podcasts-transcripts/7/K/show_7Kf4MwDBFH2QXaLZadac7W/1S9X55ZcDVk5AgA2nG0yuz.json',
  ['diaries_&_daily_life', 'relationships']),
 ('spotify-podcasts-2020/podcasts-transcripts/7/K/show_7Kf4MwDBFH2QXaLZadac7W/4d6bxJ1FtMbfo2y5kQmIZw.json',
  ['diaries_&_daily_life', 'other_hobbies']),
 ('spotify-podcasts-2020/podcasts-transcripts/7/K/show_7Kf4MwDBFH2QXaLZadac7W/1HmdZPNVswDhWvGevw5Z5j.json',
  ['diaries_&_daily_life']),
 ('spotify-podcasts-2020/podcasts-transcripts/7/K/show_7Kf4MwDBFH2QXaLZadac7W/7eLPfpTZ3MWZfuNTC4y17N.json',
  ['diaries_&_daily_life']),
 ('spotify-podcasts-2020/podcasts-transcripts/7/K/show_7Kf4MwDBFH2QXaLZadac7W/5bc22mmHX7yLd91NEwIQbV.json',
  ['diaries_&_daily_life', 'relationships']),
 ('spotify-podcasts-2020/podcasts-transcripts/7/K/show_7Kf4MwDBFH2QXaLZadac7W/1ofoG7xOVfo6WMedf7xcYH.json',
  ['diaries_&_daily_life', 'relationships']),
 ('spotify-podcasts-2020/podcasts-transcripts/7/K/show_7Kf4MwDBFH2QXaLZadac7W/6qnkY0tzalmFhjAjkkcSj9.json',
  